## **챗 모델과 프롬프트 템플릿을 사용해 간단한 LLM 애플리케이션 구축하기**

- [**언어 모델(Language Models)**](https://docs.langchain.com/docs/concepts/chat_models) 사용법  
- [**프롬프트 템플릿(Prompt Templates)**](https://docs.langchain.com/docs/concepts/prompt_templates) 사용법  

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## 언어 모델 사용하기

LangChain은 다양한 언어 모델을 지원하며, 이들을 서로 교체하여 사용할 수 있습니다.

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-5-nano", model_provider="openai")
# model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

ChatModels은 LangChain Runnables의 인스턴스로, 표준화된 인터페이스를 통해 상호작용할 수 있습니다. 모델을 간단히 호출하려면 `.invoke` 메서드에 Messages 목록을 전달하면 됩니다.

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

# 메시지 목록을 생성
messages = [
    # 시스템 메시지: 모델에게 수행할 작업이나 역할을 지시합니다.
    SystemMessage("다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."),
    # 사용자 메시지: 사용자가 모델에 보낼 실제 입력 내용입니다.
    HumanMessage("What is LangChain?"),
]

answer = model.invoke(messages)  # `invoke` 메서드를 사용해 모델을 호출합니다.
answer

AIMessage(content='LangChain이 무엇인가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 144, 'prompt_tokens': 40, 'total_tokens': 184, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMkviJ7YI8MRtCjNlIRfdxIVVKln6', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ad7019bb-2f58-490f-9b98-09ce89bdee6e-0', usage_metadata={'input_tokens': 40, 'output_tokens': 144, 'total_tokens': 184, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})

In [4]:
print(answer.content)

LangChain이 무엇인가요?


- ChatModel은 입력으로 [메시지](/docs/concepts/messages/) 객체를 받고 출력으로 메시지 객체를 생성합니다. 메시지 객체는 텍스트 내용 외에도 대화의 [역할](/docs/concepts/messages/#role)을 전달하며, [도구 호출](/docs/concepts/tool_calling/) 및 토큰 사용량과 같은 중요한 데이터를 포함합니다.  

- LangChain은 문자열이나 [OpenAI 형식](/docs/concepts/messages/#openai-format)을 통해 채팅 모델 입력을 지원합니다. 다음 예제는 모두 동일한 기능을 수행합니다:

```python
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])
```

In [7]:
# 내부적으로 HumanMessage("Hello") 로 감쌈
answer = model.invoke("안녕")
print(answer.content)

# OpenAI 형식
answer = model.invoke([{"role": "user", "content": "안녕"}])
print(answer.content)

# langChain 형식
answer = model.invoke([HumanMessage("안녕")])
print(answer.content)

안녕! 반가워요. 무엇을 도와줄까요? 필요하면 바로 도와드릴게요. 예를 들면:

- 정보 찾기나 요약
- 글쓰기나 편집 도움
- 코딩/수학 문제 해결
- 번역이나 문장 다듬기
- 한국어/영어 대화 연습

어떤 걸 도와드릴지 말해줘요.
안녕! 반가워. 오늘은 무엇을 도와줄까? 궁금한 점이 있거나 대화를 원하면 주제를 말해줘. 예를 들어 정보 찾기, 글쓰기 도움, 번역, 공부 자료, 간단한 대화 연습 등 어떤 게 필요해?
안녕하세요! 무엇을 도와드릴까요?

혹시 아래처럼 해볼까요?
- 정보 검색 및 요약
- 글쓰기 도움(교정, 아이디어, 구조화)
- 번역이나 언어 학습
- 코딩 문제 해결
- 아이디어 브레인스토밍

원하시는 주제나 질문을 말씀해 주시면 바로 도와드릴게요.


In [6]:
# 메시지 목록을 생성
batch_messages = [
    [
        SystemMessage("다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."),
        HumanMessage("What is LangChain?")
    ],
    [
        SystemMessage("다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."),
        HumanMessage("How does LangChain work?")
    ],
    [
        SystemMessage("다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."),
        HumanMessage("What are the key features of LangChain?")
    ]
]

# `model.batch()`을 사용하여 여러 개의 메시지를 한 번에 처리
# LangChain은 각 입력을 독립된 invoke() 호출처럼 처리
answers = model.batch(batch_messages)
answers

[AIMessage(content='LangChain이란 무엇입니까?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 273, 'prompt_tokens': 40, 'total_tokens': 313, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CMl0KE8O7yGZJKmEHavfV4mBSIENZ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--87df26c5-d44c-4d7b-a65c-777046395d99-0', usage_metadata={'input_tokens': 40, 'output_tokens': 273, 'total_tokens': 313, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 256}}),
 AIMessage(content='LangChain은 어떻게 작동합니까?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 210, 'prompt_tokens': 41, 'total_tokens': 251, 

In [8]:
# 결과 출력
for idx, ans in enumerate(answers):
    print(f"번역 {idx + 1}: {ans.content}")

번역 1: LangChain이란 무엇입니까?
번역 2: LangChain은 어떻게 작동합니까?
번역 3: LangChain의 주요 기능은 무엇입니까?


## 프롬프트 템플릿 (Prompt Templates)

우리는 메시지 목록을 언어 모델에 직접 전달합니다.  일반적으로 이 목록은 사용자 입력과 애플리케이션 로직의 조합으로 구성됩니다. 애플리케이션 로직은 사용자 입력(raw user input)을 받아 언어 모델에 전달할 메시지 목록으로 변환합니다. 일반적인 변환 과정에는 시스템 메시지를 추가하거나 사용자 입력을 템플릿에 맞게 포맷하는 작업이 포함됩니다.  

[프롬프트 템플릿](/docs/concepts/prompt_templates/)은 LangChain에서 이러한 변환을 돕기 위해 설계된 개념입니다. 프롬프트 템플릿은 사용자 입력(raw user input)을 받아 언어 모델에 전달할 준비가 된 데이터(프롬프트)로 반환합니다.

프롬프트 템플릿은 두 개의 사용자 변수를 입력으로 받습니다:

- **`language`**: 번역할 대상 언어  
- **`text`**: 번역할 텍스트  

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

system_template = "다음을 영어에서 {language}로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요."

# 튜플 방식 
prompt_template = ChatPromptTemplate.from_messages(
    [('system', system_template), 
     ('user', '{text}')]
)
prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='다음을 영어에서 {language}로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [11]:
# 메시지 객체 방식
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template("{text}")
])

prompt_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='다음을 영어에서 {language}로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

----------
`ChatPromptTemplate`은 하나의 템플릿에서 여러 메시지 역할을 지원합니다. `language` 매개변수는 시스템 메시지에 포맷되며, 사용자의 `text`는 사용자 메시지에 포맷됩니다.  
이 프롬프트 템플릿의 입력은 dictionary 입니다.

In [12]:
prompt = prompt_template.invoke({"language": "한국어", "text": "What is LangChain?"})

prompt

ChatPromptValue(messages=[SystemMessage(content='다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is LangChain?', additional_kwargs={}, response_metadata={})])

두 개의 메시지로 구성된 `ChatPromptValue`를 반환하는 것을 볼 수 있습니다. 메시지에 직접 액세스하려면 다음을 수행합니다.

In [13]:
prompt.to_messages()

[SystemMessage(content='다음을 영어에서 한국어로 번역하세요. 상세한 설명 말고 단순히 번역만 하세요.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is LangChain?', additional_kwargs={}, response_metadata={})]

마지막으로, 포맷된 프롬프트에서 채팅 모델을 호출할 수 있습니다.

In [14]:
response = model.invoke(prompt)
print(response.content)

LangChain이 무엇인가요?



프롬프트 템플릿은 사용자 입력과 매개변수를 언어 모델에 대한 지침으로 변환하는 데 도움을 줍니다. 이를 통해 모델이 컨텍스트를 이해하고 관련성 있고 일관된 언어 기반 출력을 생성하도록 유도할 수 있습니다.  

프롬프트 템플릿은 **딕셔너리** 형태의 입력을 받습니다. 여기서 각 키(key)는 프롬프트 템플릿에서 채워야 할 변수를 나타냅니다.  

프롬프트 템플릿은 **PromptValue**를 출력합니다. 이 **PromptValue**는 LLM 또는 ChatModel에 전달될 수 있으며, 문자열(string) 또는 메시지 목록(list of messages)으로 변환(cast)될 수도 있습니다. **PromptValue**가 존재하는 이유는 문자열과 메시지 형식 간 전환을 쉽게 하기 위함입니다.  

### 프롬프트 템플릿의 유형  

1. **String PromptTemplates (문자열 프롬프트 템플릿)**  
   - 이 프롬프트 템플릿은 단일 문자열을 포맷하는 데 사용되며, 일반적으로 더 간단한 입력에 사용됩니다.  
   - 예를 들어, 프롬프트 템플릿을 구성하고 사용하는 일반적인 방법은 다음과 같습니다:  

In [15]:
from langchain_core.prompts import PromptTemplate

template = "{topic}에 대한 농담을 하나 해 주세요"

# chain 작성 & invoke
prompt_template = PromptTemplate.from_template(template)

chain = prompt_template | model

print(chain.invoke({"topic": "고양이"}).content)

고양이가 직장을 찾으면 뭐라고 부를까요? 사무캣! 

원하시면 다른 고양이 농담도 하나 더 해드릴게요.


2. **채팅 프롬프트 템플릿(ChatPromptTemplates)**
    - 이 프롬프트 템플릿은 **메시지 목록을 포맷**하는 데 사용됩니다. 이러한 "템플릿"은 자체적으로 여러 개의 템플릿 목록으로 구성됩니다.  
    - 예를 들어, **ChatPromptTemplate**를 구성하고 사용하는 일반적인 방법은 다음과 같습니다:

In [16]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 조수입니다"),
    ("user", "{topic}에 대한 농담을 하나 해 주세요")
])

message = prompt_template.invoke({"topic": "고양이"})
message

ChatPromptValue(messages=[SystemMessage(content='당신은 도움이 되는 조수입니다', additional_kwargs={}, response_metadata={}), HumanMessage(content='고양이에 대한 농담을 하나 해 주세요', additional_kwargs={}, response_metadata={})])

In [17]:
print(model.invoke(message).content)

고양이가 컴퓨터 앞에 앉아 마우스를 쫓고 있었대요. 친구가 “무슨 일 있나요?” 하고 물으니 고양이가 말하더라고요: “여긴 마우스가 아니라 캣 포인터를 클릭하는 곳이야!”


-----------------------

위의 예에서 ChatPromptTemplate은 호출 시 두 개의 메시지를 구성합니다. 첫 번째는 시스템 메시지로, 포맷할 변수가 없습니다. 두 번째는 HumanMessage로, 사용자가 전달하는 topic 변수로 포맷됩니다.

## 메시지 플레이스홀더(MessagesPlaceholder)  

이 프롬프트 템플릿은 특정 위치에 **메시지 목록을 추가**하는 역할을 합니다.  

위의 **ChatPromptTemplate** 예제에서 두 개의 메시지를 문자열로 각각 포맷하는 방법을 확인했습니다. 하지만 특정 위치에 사용자로부터 받은 **메시지 목록**을 삽입하고 싶다면 어떻게 해야 할까요?  

이럴 때 **MessagesPlaceholder**를 사용합니다.

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 도움이 되는 조수입니다"),
    MessagesPlaceholder("msgs")
])

message = prompt_template.invoke({"msgs": [HumanMessage(content="고양이에 대한 농담을 하나 해 주세요")]})
message

ChatPromptValue(messages=[SystemMessage(content='당신은 도움이 되는 조수입니다', additional_kwargs={}, response_metadata={}), HumanMessage(content='고양이에 대한 농담을 하나 해 주세요', additional_kwargs={}, response_metadata={})])

In [19]:
print(model.invoke(message).content)

하나 드릴게요: 왜 고양이는 컴퓨터를 잘 쓸까요? 마우스가 있어서요.


---------------------------
이렇게 하면 두 개의 메시지 목록이 생성됩니다. 첫 번째는 **시스템 메시지**이고, 두 번째는 우리가 전달한 **HumanMessage**입니다. 만약 5개의 메시지를 전달했다면, 총 6개의 메시지가 생성됩니다 (시스템 메시지 1개 + 전달된 5개 메시지).  

이 방법은 특정 위치에 **메시지 목록을 삽입**할 때 유용합니다.  